In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import scipy as sp
import re
import os
from itertools import groupby
import datetime as dt

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster', rc={'font.size':35,
                              'axes.titlesize':50,
                              'axes.labelsize':35})

pd.set_option('display.max_rows', 100000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.width', 100000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

### Variable Definitions

- A   = active lever (1 = Left, any other number = Right)
- B   = number of right nose pokes
- C   = number of left nose pokes
#### D   = event time stamp
#### E   = event identity stamp

### Event Stamps

####      46 - Right lever extends 
- trial starts

####      47 - Left lever extends
- trial starts

####      12 - Intertrial interval begins
####      13 - Intertrial interval over and reinforcement available again

####      10 - ITI Right lever press
####      11 - ITI Left lever press
####      14 - ITI Head entry

####      40 - PR reset lever pressed (right)
####      41 - PR reset lever pressed (left)
####      3  - Non-ITI Left lever press
####       4  - Non-ITI Right lever press
####      7  - Non-ITI Head entry
####      17 - Pellet delivery

-      5  - Reinforced left lever press
-      6  - Reinforced right lever press
-      8  - Non-ITI Right nose poke
-      9  - Non-ITI Left nose poke
-      15 - ITI Right nose-poke
-      16 - ITI Left nose-poke
-      18 - Tone delivery
-      42 - Right cue light active
-      43 - Left cue light active
-      44 - Right cue light off
-      45 - Left cue light off
-      48 - Right lever retracts
-      49 - Left lever retracts
-      50 - Head entry to initiate trial
-     100 - End of session marker

In [2]:
def get_file_info(file_path_name):
    # takes in a file path, finds the date, animal_number, session, session number, and sub_session (if subsession=True),
    #saves each accordingly
    
    path_split = file_path_name.split('/')
    file_split = path_split[-1].split('_')

    for row in file_split:

        if re.search(r'^\d\d\d\d\d\d$', row):
            date = row
        if re.search(r'^\d\d\d$', row):
            animal_number = row
        if re.search(r'^\d$', row):
            session_num = row 
        
    try:
        #session = str(file_split[0] + '_' + session_num)
        session = file_split[0]
    except: 
        session = file_split[0]

    return date, animal_number, session, session_num

In [3]:
def make_array_from_path(file_path_name):
    # takes in file path, creates array (each row of array is a row from original med file)
    import numpy as np
    import pandas as pd
    
    data = pd.read_table(file_path_name)
    data_table = pd.DataFrame(data = data)
    data_array = data_table.values
    
    return data_array

In [4]:
def get_events_and_times(array, letter_1, letter_2, order):
    #takes in array and the letters Med PC program uses to signify an event_stamp and time_stamp; 
    #finds all event_stamps and corresponding time_stamps; creats pandas dataframe
    #order is whether the event or time stamp is letter_1 or letter_2
    
    import re
    import numpy as np
    import pandas as pd

    i = 0
    letter_1_array = []
    letter_2_array = []

    while i < len(array):
        if re.search(str(letter_1 + ':'), str(array[i])):
            i = i + 1
            while re.search('\[\'\ ', str(array[i])):
                split = array[i][0].split()[1:]
                for element in split:
                    letter_1_array.append(float(element))
                i = i + 1
        if re.search(str(letter_2 + ':'), str(array[i])):
            i = i + 1
            while re.search('\[\'\ ', str(array[i])):
                split = array[i][0].split()[1:]
                for element in split:
                    letter_2_array.append(float(element))
                i = i + 1
        i = i + 1
    
    if order == 'event':
        eandt = np.column_stack((letter_1_array, letter_2_array))
        es_et = pd.DataFrame(data = eandt)
        es_et.columns = ['event_stamp', 'time_stamp']
        
    if order == 'time':
        eandt = np.column_stack((letter_2_array, letter_1_array))
        es_et = pd.DataFrame(data = eandt)
        es_et.columns = ['event_stamp', 'time_stamp']
    
    return es_et

In [5]:
def get_trial_times_PR(events_and_times, right_extends, left_extends, ITI_start, ITI_end, session_end):
    #takes in dataframe of events and corresponding time_stamps and the Med PC program code for start and end of trial segment
    #e.g. conditioned stimulus onset and offset, ITI onset and offset - here both levers extend so can just use right lever as start
    #finds time_stamps for each start and end, puts into new dataframe
    import numpy as np
    import pandas as pd

    start_time = events_and_times[(events_and_times.event_stamp == right_extends)]['time_stamp'].values
    end_time = events_and_times[(events_and_times.event_stamp == ITI_start) | (events_and_times.event_stamp == session_end)]['time_stamp'].values
    start_time_ITI = events_and_times[events_and_times.event_stamp == ITI_start]['time_stamp'].values
    end_time_ITI = events_and_times[events_and_times.event_stamp == ITI_end]['time_stamp'].values
    
    times = pd.DataFrame(data = [start_time, end_time, start_time_ITI, end_time_ITI])
    times = times.T
    times.columns = ['trial_start', 'trial_end', 'ITI_start', 'ITI_end']
    
    return times

In [6]:
def create_trial_df(events_and_times, trial_times, trial_type):
    #takes in df of events and times and df of trial times
    #creates tidy df containing events and times according to trials
    i = 0
    trial_data = pd.DataFrame()
    
    if trial_type == 'ITI':
        while i < len(trial_times):
            data = events_and_times[(events_and_times['time_stamp'] >= trial_times['ITI_start'][i]) & (events_and_times['time_stamp'] <= trial_times['ITI_end'][i])]
            data['trial_#'] = [i]*len(data)
            data['ITI_start'] = trial_times['ITI_start'][i]
            data['ITI_end'] = trial_times['ITI_end'][i]
            trial_data = pd.concat([trial_data, data], axis = 0, ignore_index=True)
            i = i + 1
    elif trial_type == 'within':
        while i < len(trial_times):
            data = events_and_times[(events_and_times['time_stamp'] >= trial_times['trial_start'][i]) & (events_and_times['time_stamp'] <= trial_times['trial_end'][i])]
            data['trial_#'] = [i]*len(data)
            data['trial_start'] = trial_times['trial_start'][i]
            data['trial_end'] = trial_times['trial_end'][i]
            trial_data = pd.concat([trial_data, data], axis = 0, ignore_index=True)
            i = i + 1
    
    return trial_data

In [7]:
def count(trial_data, event_num):
    #get count of event_num
    trials = len((trial_data['trial_#'].unique()))
    i = 0
    count = []
    while i < trials:
        data = trial_data[trial_data['trial_#'] == i]
        count.append(len(data[data['event_stamp'] == event_num]))
        i = i + 1
    
    return count

In [8]:
def latency(trial_data, event_num, trial_type):
    #get latency of time between trial_start and event_num
    trials = len((trial_data['trial_#'].unique()))
    i = 0
    latency_1st = []
    latency_ave = []
    while i < trials:
        data = trial_data[trial_data['trial_#'] == i]
        data = data[data['event_stamp'] == event_num]
        if len(data) < 1:
            latency_1st.append(np.nan)
            latency_ave.append(np.nan)
        else:
            if trial_type == 'ITI':
                latency_1st.append((data['time_stamp'] - data['ITI_start']).values[0]/100)
                latency_ave.append((data['time_stamp'] - data['ITI_start']).values.mean()/100)
            elif trial_type == 'within':
                latency_1st.append((data['time_stamp'] - data['trial_start']).values[0]/100)
                latency_ave.append((data['time_stamp'] - data['trial_start']).values.mean()/100)
        i = i + 1
    
    return latency_1st, latency_ave

In [9]:
def create_session_df(trial_data, event_dic, trial_type):
    
    trials = len((trial_data['trial_#'].unique()))
    
    #get times
    if trial_type == 'ITI':
        times_df = pd.DataFrame(data = [trial_data.groupby('trial_#')['ITI_start'].unique(), trial_data.groupby('trial_#')['ITI_end'].unique()])
        times_df = times_df.T
    elif trial_type == 'within':
        times_df = pd.DataFrame(data = [trial_data.groupby('trial_#')['trial_start'].unique(), trial_data.groupby('trial_#')['trial_end'].unique()])
        times_df = times_df.T  
        
    i = 0
    trial_times = []

    while i < times_df.shape[0]:
        if trial_type == 'ITI':
            trial_time = (times_df.iloc[i]['ITI_end'] - times_df.iloc[i]['ITI_start'])/100
        elif trial_type == 'within':
            trial_time = (times_df.iloc[i]['trial_end'] - times_df.iloc[i]['trial_start'])/100
        trial_times.append(trial_time[0])
        i += 1
    
    session_data = pd.DataFrame(index=np.arange(0,trials))

    for name, event in event_dic.items():
        session_data[str(name + '_count')] = count(trial_data, event)
        session_data[str(name + '_latency_1st')], session_data[str(name + '_latency_ave')] = latency(trial_data, event, trial_type)
    
    #add bin cout (6 bins of 10 min each)
    if trial_type == 'ITI':
        session_data['trial_bin'] = pd.cut(times_df['ITI_end'].str[0].values/100, bins=6, labels=False)
    elif trial_type == 'within':
        session_data['trial_bin'] = pd.cut(times_df['trial_end'].str[0].values/100, bins=6, labels=False)
    
    session_data['trial_num'] = np.arange(session_data.shape[0])
    session_data['trial_duration'] = trial_times
    
    return session_data

### Single directory run

In [10]:
path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round1/switch_lever/reinst_drug/session'

file_names = os.listdir(path)
print(len(file_names))

file_paths = []

for file in file_names:
    
    file_path = path + '/' + file
    file_paths.append(file_path)

print(len(file_paths))
file_paths[0]

17
17


'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round1/switch_lever/reinst_drug/session/RDP_1_825_200323'

### Multiple directory run

In [11]:
path_dir_orig = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round1/orig_lever'

dir_list_orig = os.listdir(path_dir_orig)
print(len(dir_list_orig))
dir_list_orig

3


['FR_retrain', 'FR_train', 'PR']

In [ ]:
file_paths_orig = []

for name in dir_list_orig:
    print(name)
    
    new_path = path_dir_orig + '/' + name
    new_dir_list = os.listdir(new_path)
    print(new_path)
    print(len(new_dir_list), '\n')
    
    for file in new_dir_list:
        file_path = new_path + '/' + file
        file_paths_orig.append(file_path)

print(len(file_paths_orig))
file_paths_orig[0]

### Read in animal data

In [11]:
#read animal info (lever assignment, group etc) into df

path_animal_info = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/assignments/PR_r2_2020Feb.xlsx'
    
animal_info = pd.read_excel(path_animal_info)
animal_info = pd.DataFrame(data = animal_info)

animal_info.head()

,cage,animal,group,lever
0,158,825,1,0
1,158,826,1,0
2,158,827,1,0
3,159,828,1,1
4,159,829,1,1


### Process data

In [12]:
%%time
final_data = pd.DataFrame()

bit_dic = {'reinforcement': 17, 'right_lever': 4, 'left_lever': 3, 'HE': 7}
ITI_dic = {'reinforcement': 17, 'right_lever': 10, 'left_lever': 11, 'HE': 14}

for file in file_paths:

    print(file) 
    
    #proccess file
    date, animal_number, session, session_num = get_file_info(file)
    array = make_array_from_path(file)
    events_and_times = get_events_and_times(array, "D", "E", 'time')
    trial_times = get_trial_times_PR(events_and_times, 46, 47, 12, 13, 100)
    trial_data = create_trial_df(events_and_times, trial_times, 'within')
    ITItrial_data = create_trial_df(events_and_times, trial_times, 'ITI')
    session_data = create_session_df(trial_data, bit_dic, 'within')
    ITIsession_data = create_session_df(ITItrial_data, ITI_dic, 'ITI')
   
    #create pandas tidy df for within session
    session_data['type'] = ['session']*(session_data.shape[0])    
    session_data['date'] = [date]*(session_data.shape[0])
    session_data['animal_number'] = [animal_number]*(session_data.shape[0])
    session_data['session'] = [session]*(session_data.shape[0])
    session_data['session_num'] = [session_num]*(session_data.shape[0])
    session_data['active_lever'] = int(animal_info[animal_info['animal'] == int(animal_number)]['lever'])
    session_data['group'] = int(animal_info[animal_info['animal'] == int(animal_number)]['group'])
    session_data['cage'] = int(animal_info[animal_info['animal'] == int(animal_number)]['cage'])
   
    session_data = session_data[['type', 'date', 'session', 'session_num', 'cage', 'animal_number', 'group', 'active_lever', 
                                 'trial_num', 'trial_bin', 'trial_duration', 'reinforcement_count', 'reinforcement_latency_1st', 
                                 'HE_count', 'HE_latency_1st', 'HE_latency_ave',
                                 'right_lever_count', 'right_lever_latency_1st',
                                 'left_lever_count', 'left_lever_latency_1st',
                                 'reinforcement_latency_ave', 'right_lever_latency_ave', 'left_lever_latency_ave']]
    
    #create pandas tidy df for within ITI
    ITIsession_data['type'] = ['ITI']*(ITIsession_data.shape[0])  
    ITIsession_data['date'] = [date]*(ITIsession_data.shape[0])
    ITIsession_data['animal_number'] = [animal_number]*(ITIsession_data.shape[0])
    ITIsession_data['session'] = [session]*(ITIsession_data.shape[0])
    ITIsession_data['session_num'] = [session_num]*(ITIsession_data.shape[0])
    ITIsession_data['active_lever'] = int(animal_info[animal_info['animal'] == int(animal_number)]['lever'])
    ITIsession_data['group'] = int(animal_info[animal_info['animal'] == int(animal_number)]['group'])
    ITIsession_data['cage'] = int(animal_info[animal_info['animal'] == int(animal_number)]['cage'])
    
    ITIsession_data = ITIsession_data[['type', 'date', 'session', 'session_num', 'cage', 'animal_number', 'group', 'active_lever', 
                                 'trial_num', 'trial_bin', 'trial_duration', 'reinforcement_count', 'reinforcement_latency_1st', 
                                 'HE_count', 'HE_latency_1st', 'HE_latency_ave',
                                 'right_lever_count', 'right_lever_latency_1st',
                                 'left_lever_count', 'left_lever_latency_1st',
                                 'reinforcement_latency_ave', 'right_lever_latency_ave', 'left_lever_latency_ave']]

    #add to final data frame
    final_data = pd.concat([final_data, session_data, ITIsession_data], axis = 0, ignore_index=True)

final_data.head()

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round1/switch_lever/reinst_drug/session/RDP_1_825_200323
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round1/switch_lever/reinst_drug/session/RDP_1_826_200323
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round1/switch_lever/reinst_drug/session/RDP_1_827_200323
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round1/switch_lever/reinst_drug/session/RDP_1_828_200323
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round1/switch_lever/reinst_drug/session/RDP_1_829_200323
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round1/switch_lever/reinst_drug/session/RDP_1_830_200323
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round1/switch_lever/reinst_drug/session/RDP_1_831_200323

,type,date,session,session_num,cage,animal_number,group,active_lever,trial_num,trial_bin,trial_duration,reinforcement_count,reinforcement_latency_1st,HE_count,HE_latency_1st,HE_latency_ave,right_lever_count,right_lever_latency_1st,left_lever_count,left_lever_latency_1st,reinforcement_latency_ave,right_lever_latency_ave,left_lever_latency_ave
0,session,200323,RDP,1,158,825,1,0,0,0,2.15,0,NaN,0,NaN,NaN,0,NaN,1,2.14,NaN,NaN,2.14
1,session,200323,RDP,1,158,825,1,0,1,0,102.92,0,NaN,1,78.71,78.710,12,55.23,1,102.91,NaN,68.805833,102.91
2,session,200323,RDP,1,158,825,1,0,2,0,22.81,0,NaN,0,NaN,NaN,0,NaN,1,22.80,NaN,NaN,22.80
3,session,200323,RDP,1,158,825,1,0,3,0,240.28,0,NaN,10,0.13,135.228,0,NaN,1,240.27,NaN,NaN,240.27
4,session,200323,RDP,1,158,825,1,0,4,0,8.36,0,NaN,0,NaN,NaN,0,NaN,1,8.35,NaN,NaN,8.35


In [14]:
#make new df with combined data
df_final = final_data.copy()

lp_train_df_trial = pd.DataFrame()

animals = df_final['animal_number'].unique()

for animal in animals:
    
    #get animal data
    animal_data = df_final[df_final['animal_number'] == animal]
    
    #generate session df
    sessions = animal_data['session'].unique()
    
    for session in sessions:
        #get session data for animal
        animal_data_session = animal_data[animal_data['session'] == session]
        
        #compute and fill session data
        if animal_data_session['active_lever'].unique() == 1:
            animal_data_session['active_lever_count'] = animal_data_session['right_lever_count']
            animal_data_session['active_lever_latency_1st'] = animal_data_session['right_lever_latency_1st']
            animal_data_session['active_lever_latency_ave'] = animal_data_session['right_lever_latency_ave']
            animal_data_session['inactive_lever_count'] = animal_data_session['left_lever_count']
            animal_data_session['inactive_lever_latency_1st'] = animal_data_session['left_lever_latency_1st']
            animal_data_session['inactive_lever_latency_ave'] = animal_data_session['left_lever_latency_ave']

        elif animal_data_session['active_lever'].unique() == 0:
            animal_data_session['active_lever_count'] = animal_data_session['left_lever_count']
            animal_data_session['active_lever_latency_1st'] = animal_data_session['left_lever_latency_1st']
            animal_data_session['active_lever_latency_ave'] = animal_data_session['left_lever_latency_ave']
            animal_data_session['inactive_lever_count'] = animal_data_session['right_lever_count']
            animal_data_session['inactive_lever_latency_1st'] = animal_data_session['right_lever_latency_1st']
            animal_data_session['inactive_lever_latency_ave'] = animal_data_session['right_lever_latency_ave']
            
        lp_train_df_trial = lp_train_df_trial.append(animal_data_session)
        
lp_train_df_trial.reset_index(inplace=True, drop=True)    

lp_train_df_trial = lp_train_df_trial[['type', 'date', 'session', 'session_num', 'cage', 'animal_number', 'group',
                                       'active_lever', 'trial_num', 'trial_bin', 'trial_duration',
                                       'reinforcement_count', 'reinforcement_latency_1st',
                                       'HE_count', 'HE_latency_1st', 'HE_latency_ave',
                                       'active_lever_count', 'active_lever_latency_1st',
                                       'inactive_lever_count', 'inactive_lever_latency_1st', 
                                       'active_lever_latency_ave', 'inactive_lever_latency_ave', 'reinforcement_latency_ave']]

lp_train_df_trial['lever_ratio'] = lp_train_df_trial['active_lever_count'] / (lp_train_df_trial['active_lever_count'] + lp_train_df_trial['inactive_lever_count'])

lp_train_df_trial['stage'] = 'reinst_drug'

print(lp_train_df_trial.shape)
lp_train_df_trial.head()

(1007, 25)


,type,date,session,session_num,cage,animal_number,group,active_lever,trial_num,trial_bin,trial_duration,reinforcement_count,reinforcement_latency_1st,HE_count,HE_latency_1st,HE_latency_ave,active_lever_count,active_lever_latency_1st,inactive_lever_count,inactive_lever_latency_1st,active_lever_latency_ave,inactive_lever_latency_ave,reinforcement_latency_ave,lever_ratio,stage
0,session,200323,RDP,1,158,825,1,0,0,0,2.15,0,NaN,0,NaN,NaN,1,2.14,0,NaN,2.14,NaN,NaN,1.000000,reinst_drug
1,session,200323,RDP,1,158,825,1,0,1,0,102.92,0,NaN,1,78.71,78.710,1,102.91,12,55.23,102.91,68.805833,NaN,0.076923,reinst_drug
2,session,200323,RDP,1,158,825,1,0,2,0,22.81,0,NaN,0,NaN,NaN,1,22.80,0,NaN,22.80,NaN,NaN,1.000000,reinst_drug
3,session,200323,RDP,1,158,825,1,0,3,0,240.28,0,NaN,10,0.13,135.228,1,240.27,0,NaN,240.27,NaN,NaN,1.000000,reinst_drug
4,session,200323,RDP,1,158,825,1,0,4,0,8.36,0,NaN,0,NaN,NaN,1,8.35,0,NaN,8.35,NaN,NaN,1.000000,reinst_drug


In [15]:
lp_train_df_trial.to_csv('reinst_drug.csv', index=False)

In [21]:
file_path = file_paths[1]
print('file_path:', '\n', file_path, '\n')

date, animal_number, session, session_num = get_file_info(file_path)
print('date:', date, 'animal_number:', animal_number, 'session:', session, 'session_num:', session_num, '\n')

array = make_array_from_path(file_path)
print(array[0:5], '\n')

events_and_times = get_events_and_times(array, "D", "E", 'time')
print(events_and_times.head(), '\n')

trial_times = get_trial_times_PR(events_and_times, 46,47,12,13,100)
print(trial_times.head(), '\n')

trial_data = create_trial_df(events_and_times, trial_times, 'within')
print(trial_data.head(), '\n')

ITI_trial_data = create_trial_df(events_and_times, trial_times, 'ITI')
print(ITI_trial_data.head(), '\n')

bit_dic = {'reinforcement': 17, 'right_lever_choice': 4, 'left_lever_choice': 3, 'HE_trial': 7}
session_data = create_session_df(trial_data, bit_dic, 'within')

ITI_dic = {'right_lever_choice': 10, 'left_lever_choice': 11, 'HE_trial': 14}
ITI_data = create_session_df(ITI_trial_data, ITI_dic, 'ITI')

file_path: 
 C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/reward_learning_tc/round1/orig_lever/PR_test/PR13_1_826_200228 

date: 200228 animal_number: 826 session: PR13 session_num: 1 

[['Start Date: 02/28/20']
 ['End Date: 02/28/20']
 ['Subject: 826']
 ['Experiment: 0']
 ['Group: 0']] 

   event_stamp  time_stamp
0         42.0         0.0
1         46.0         0.0
2         47.0         0.0
3          4.0       184.0
4          4.0       256.0 

   trial_start  trial_end  ITI_start  ITI_end
0          0.0      280.0      280.0   3279.0
1       3280.0     3553.0     3553.0   6552.0
2       6553.0     6707.0     6707.0   9706.0
3       9707.0    10002.0    10002.0  13001.0
4      13002.0    13562.0    13562.0  16561.0 

   event_stamp  time_stamp  trial_#  trial_start  trial_end
0         42.0         0.0        0          0.0      280.0
1         46.0         0.0        0          0.0      280.0
2         47.0         0.0        0          0.0      280.0
3     

### Event Stamps

####      46 - Right lever extends 
- trial starts

####      47 - Left lever extends
- trial starts

####      12 - Intertrial interval begins
####      13 - Intertrial interval over and reinforcement available again

####      10 - ITI Right lever press
####      11 - ITI Left lever press
####      14 - ITI Head entry

####      40 - PR reset lever pressed (right)
####      41 - PR reset lever pressed (left)
####      3  - Non-ITI Left lever press
####       4  - Non-ITI Right lever press
####      7  - Non-ITI Head entry
####      17 - Pellet delivery

-      5  - Reinforced left lever press
-      6  - Reinforced right lever press
-      8  - Non-ITI Right nose poke
-      9  - Non-ITI Left nose poke
-      15 - ITI Right nose-poke
-      16 - ITI Left nose-poke
-      18 - Tone delivery
-      42 - Right cue light active
-      43 - Left cue light active
-      44 - Right cue light off
-      45 - Left cue light off
-      48 - Right lever retracts
-      49 - Left lever retracts
-      50 - Head entry to initiate trial
-     100 - End of session marker